In [1]:
import syft as sy
import os
from syft import ActionObject
from collections import defaultdict

Start this using

```
hagrid launch domain to docker:8080 --dev --verbose
```

In [2]:
client = sy.login(url="http://localhost:8080", email="info@openmined.org", password="changethis")

Logged into <determined_altman: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# Mount storage container with Helm azure container

In [3]:
client.api.services.blob_storage.mount_azure(
    account_name='helmprojectstorage',
    container_name='helm',
    account_key=os.environ["HELM_STORAGE_ACCOUNT_KEY"],
    bucket_name='helmazurebucket',
)

SyftSuccess: Mounting Azure Successful!

In [4]:
blob_files = client.api.services.blob_storage.get_files_from_bucket(bucket_name='helmazurebucket')

In [5]:
blob_files

[syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile]

# Start workers

In [6]:
client.worker.start_workers(n=3)

SyftSuccess: 3 workers added

In [7]:
client.worker.list()

[syft.service.worker.worker_service.DockerWorker,
 syft.service.worker.worker_service.DockerWorker,
 syft.service.worker.worker_service.DockerWorker]

# Create Dataset

In [8]:
train_file = sy.ActionObject.from_path("short_input.jsonl").send(client).syft_action_data
scenario_file = scenario_obj = sy.ActionObject.from_path(path="scenario_data.jsonl").send(client).syft_action_data

In [9]:
# train_file = [f for f in blob_files if "train-00" in f.file_name][0]
# scenario_file = [f for f in blob_files if "scenario_data" in f.file_name][0]

In [10]:
helm_dataset = sy.Dataset(
    name="Helm Dataset",
    asset_list=[
        sy.Asset(
            name="helm train data",
            data=ActionObject.from_obj([train_file]),
            mock=sy.ActionObject.empty()
        ),
        sy.Asset(
            name="helm test data",
            data=ActionObject.from_obj([scenario_file]),
            mock=sy.ActionObject.empty()
        )
    ]
)

In [11]:
client.upload_dataset(helm_dataset)

SyftWarning: You're approving a request on high side domain which may host datasets with private information.

Would you like to proceed? [y/n]: y


  0%|                                                                                                                 | 0/2 [00:00<?, ?it/s]

Uploading: helm train data


 50%|████████████████████████████████████████████████████▌                                                    | 1/2 [00:00<00:00,  1.25it/s]

Uploading: helm test data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.31it/s]


SyftSuccess: Dataset uploaded to 'determined_norvig'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [12]:
helm_ds = client.datasets["Helm Dataset"]
helm_train_files = helm_ds.assets["helm train data"]
helm_test_files = helm_ds.assets["helm test data"]

SyftWarning: You're performing an operation on high side domain, which could host datasets with private information.

# Syft functions

In [13]:
@sy.syft_function()
def compute_document_data_overlap(domain, scenario_file, input_files, n):
    print("starting overlap computation")

    from nltk import ngrams
    from collections import defaultdict
    from string import punctuation
    import re, json
    import time

    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    
    def create_ngram_index(light_scenarios, n_values, stats_key_counts):
        ngram_index = {n:{}  for n in n_values}
        for i, scenario in enumerate(light_scenarios):
            if i%20 == 0:
                print(f"n_gram indexing progress: {(i/len(light_scenarios))*100:.2f}%")
            for n in n_values:
                stats_key = scenario['scenario_key'] + '_' + str(n)
                stats_key_counts[stats_key] = len(scenario['instances'])
                for instance in scenario['instances']:
                    id = instance['id']                    
                    input_tokens = r.split(instance['input'].lower())
                    for input_ngram in ngrams(input_tokens, n):
                        if input_ngram not in ngram_index[n]:
                            ngram_index[n][input_ngram] = set()
                        ngram_index[n][input_ngram].add(stats_key + '+' + id + '+' + 'input')

                    # compute reference ngrams
                    for reference in instance['references']:
                        reference_unigrams = r.split(reference.lower())
                        for reference_ngram in ngrams(reference_unigrams, n):
                            if reference_ngram not in ngram_index[n]:
                                ngram_index[n][reference_ngram] = set()
                            ngram_index[n][reference_ngram].add(stats_key + '+' + id + '+' + 'references')
        return ngram_index
    
    # SETUP
    print("preparing scenarios and creating indexes")
    start = time.time()
    light_scenarios = []
    for i, (bytes_read, light_scenario_json) in enumerate(scenario_file.iter_lines(progress=True)):
        if i % 20 == 0:
            print(f"scenario creation progress: {(bytes_read/scenario_file.file_size)*100:.2f}%")

        light_scenario_dict: dict = json.loads(light_scenario_json)

        light_scenario_key_dict: dict = light_scenario_dict["scenario_key"]
        scenario_spec = str(light_scenario_key_dict["scenario_spec"])

        light_scenario_key = scenario_spec + '_' + light_scenario_key_dict["split"]
        light_instances = [
            {
                'input': instance_dict['input'], 
                'references': instance_dict['references'], 
                'id': instance_dict["id"]
            }
            for instance_dict in light_scenario_dict["instances"]
        ]
        light_scenarios.append({'scenario_key': light_scenario_key, 'instances': light_instances})
    print(f"Finished creating scenarios ({time.time()-start}s)")
    
    print("Creating indexes")
    
    start = time.time()
    stats_key_counts = defaultdict(int)
    ngram_index = create_ngram_index(
        light_scenarios=light_scenarios, n_values=[n], stats_key_counts=stats_key_counts
    )
    print(f"Finished creating indexes ({time.time()-start}s)")
        
    
    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    stats_key_to_input_ids = defaultdict(set)
    stats_key_to_reference_ids = defaultdict(set)
    print("computing overlap")
    start = time.time()
    
    domain.init_progress(input_files[0].file_size)

    for input_file in input_files:
        for i, (bytes_read, line) in enumerate(input_file.iter_lines(progress=True)):
            if i%1000 == 0:
                print(f"computing overlap progress: {(bytes_read / input_file.file_size) * 100:.2f}%")
                domain.set_progress(bytes_read)
            if i==10000:
                break
            document = json.loads(line)["text"]
            document_tokens = r.split(document.lower())
            for n in ngram_index.keys():
                for document_ngram in ngrams(document_tokens, n):
                    if document_ngram in ngram_index[n]:
                        for entry_overlap_key in ngram_index[n][document_ngram]:
                            stats_key, id, part = entry_overlap_key.split("+")
                            if part == "input":
                                stats_key_to_input_ids[stats_key].add(id)
                            elif part == "references":
                                stats_key_to_reference_ids[stats_key].add(id)
    print(f"Finished computing overlap ({time.time()-start}s)")
    print("done")
    
    return stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts

SyftSuccess: Syft function 'compute_document_data_overlap' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [14]:
client.code.submit(compute_document_data_overlap)

SyftSuccess: User Code Submitted

In [15]:
@sy.syft_function_single_use(input_files=helm_train_files, scenario_files=helm_test_files)
def main_function(domain, input_files, scenario_files):
    N = [5, 9, 13]
    jobs = []
    for n in N[:1]:
        for scenario_file in scenario_files:
            batch_job = domain.launch_job(
                compute_document_data_overlap,
                scenario_file=scenario_file,
                input_files=input_files,
                n=n
            )
            jobs.append(batch_job)

    return None


SyftSuccess: Syft function 'main_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [16]:
client.code.request_code_execution(main_function)

syft.service.request.request.Request

In [18]:
client.requests[-1].approve(approve_nested=True)

SyftWarning: You're approving a request on high side domain which may host datasets with private information.

Would you like to proceed? [y/n]: y
Request approved for domain determined_norvig


SyftSuccess: Request e7f73ae2ab8e4f1eb0efb02d676e51c1 changes applied

In [19]:
job = client.code.main_function(input_files=helm_train_files,
                                scenario_files=helm_test_files,
                                blocking=False)

# Inspect Jobs and get results

In [21]:
job

```python
class Job:
    id: UID = 4d68a345f6f34892bc7d4d42ee1f004b
    status: created
    has_parent: False
    result: ActionDataEmpty <None>
    logs:

0 
    
```

In [22]:
# job.subjobs

In [26]:
job.subjobs

[syft.service.job.job_stash.Job]

In [27]:
# job.wait().get()

In [28]:
job.subjobs[0].logs()

starting overlap computation
preparing scenarios and creating indexes
scenario creation progress: 0.90%
Finished creating scenarios (0.2474043369293213s)
Creating indexes
n_gram indexing progress: 0.00%
Finished creating indexes (0.033365488052368164s)
computing overlap
computing overlap progress: 3.83%
Finished computing overlap (0.07294917106628418s)
done




In [29]:
results = [j.wait().get() for j in job.subjobs]

In [30]:
#stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts
results

[(defaultdict(set,
              {"{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'philosophy'}}_test_5": {'id328'},
               "{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'philosophy'}}_valid_5": {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {"{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'philosophy'}}_train_5": 5,
               "{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'philosophy'}}_valid_5": 34,
               "{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'philosophy'}}_test_5": 311,
               "{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'anatomy'}}_train_5": 5,
               "{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'anatomy'}}_valid_5": 14,
               "{'class_name': 'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', 'args': {'subject': 'anatomy'}}_test_5": 135}))]

In [31]:
# results[0]

# Aggregate

In [32]:
stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts = zip(*results)

total_input_ids = defaultdict(set)
total_reference_ids = defaultdict(set)
total_stats_key_counts = defaultdict(int)

for d in stats_key_counts:
    for key, val in d.items():
        total_stats_key_counts[key] += val


for d in stats_key_to_input_ids:
    for key in d:
        new_set = set()
        if key in total_input_ids:
            new_set = total_input_ids[key]
        new_set = new_set.union(d[key])
        total_input_ids[key] = new_set

for d in stats_key_to_reference_ids:
    for key in d:
        new_set = set()
        if key in total_reference_ids:
            new_set = total_reference_ids[key]
        new_set = total_reference_ids[key].union(d[key])
        total_reference_ids[key] = new_set

all_data_overlap_stats = []
for stats_key, count in total_stats_key_counts.items():
    data_overlap_stats = {
        'data_overlap_stats_key': None,
        'num_instances': count,
        'instance_ids_with_overlapping_input': sorted(total_input_ids[stats_key]),
        'instance_ids_with_overlapping_reference': sorted(total_reference_ids[stats_key]),
    }
    subject, split, n_str = stats_key.rsplit('_', 2)
    data_overlap_stats['data_overlap_stats_key'] = {
        'light_scenario_key': {'scenario_spec': subject, 'split': split},
        'overlap_protocol_spec': {'n': int(n_str)}
    }
    all_data_overlap_stats.append(data_overlap_stats)


In [33]:
from pprint import pprint
pprint(all_data_overlap_stats)

[{'data_overlap_stats_key': {'light_scenario_key': {'scenario_spec': "{'class_name': "
                                                                     "'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', "
                                                                     "'args': "
                                                                     "{'subject': "
                                                                     "'philosophy'}}",
                                                    'split': 'train'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': [],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 5},
 {'data_overlap_stats_key': {'light_scenario_key': {'scenario_spec': "{'class_name': "
                                                                     "'helm.benchmark.scenarios.mmlu_scenario.MMLUScenario', "
                                                                    